In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/WaterBody_Extraction/

/content/drive/MyDrive/WaterBody_Extraction


In [7]:
%ls
!unzip CN_tiles.zip
!unzip BZ_tiles.zip -d BZ_tiles

BZ_tiles.zip  CN_tiles.zip  loss.py  main.ipynb  metrics.py  tif_processor.py


In [4]:
!pip install pytorch-lightning
!pip install segmentation-models-pytorch
!pip install torchmetrics
!pip install rasterio
!pip install gdal

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 59.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 60.3 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=472c9d11b83d6c882edea3e30b86d0773e529064d449992e7a4e677391413b2e
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=c5d3a633d01ead4eeead90f193849a3d7f4dcca2dc037647

In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.nn.functional import  binary_cross_entropy_with_logits, sigmoid
import pytorch_lightning as pl
import torchmetrics
from torchmetrics import Metric
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
import segmentation_models_pytorch as smp
import numpy as np
import random

In [6]:
def seed_everything(seed=11):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything(11)

# Import from self-written code

In [7]:
from tif_processor import SatelliteDataset
from metrics import MIoU
from loss import dice_bce_loss_with_logits, dice_loss_with_logits

In [8]:
%ls

BZ_tiles/     CN_tiles/     loss.py     metrics.py    tif_processor.py
BZ_tiles.zip  CN_tiles.zip  main.ipynb  __pycache__/


# Data Path

In [9]:
# Training dataset
feature_tiles_train = "CN_tiles/tiles/features"
label_tiles_train = "CN_tiles/tiles/labels"

# Validation dataset
feature_tiles_val = "CN_tiles/tiles/validation/features"
label_tiles_val = "CN_tiles/tiles/validation/labels"

# Test dataset
feature_tiles_test = "BZ_tiles/tiles/features"
label_tiles_test = "BZ_tiles/tiles/labels"


# HyperParameters

In [19]:
batch_size=4
shuffle=True
EPOCHS=50

# Dataset & DataLoader

In [11]:
train_dataset= SatelliteDataset(
feature_dir=feature_tiles_train,
label_dir=label_tiles_train,
weight_dir=None,
tiles=range(0, 1440),
mu=None,
sigma=None,
sample=None
)
val_dataset= SatelliteDataset(
feature_dir=feature_tiles_val,
label_dir=label_tiles_val,
weight_dir=None,
tiles=range(1440, 1600),
mu=None,
sigma=None,
sample=None
)

In [12]:
from torch.utils.data import random_split, DataLoader

# create DataLoader
train_loader = DataLoader(train_dataset, batch_size, shuffle)
val_loader = DataLoader(val_dataset, batch_size, shuffle)
len(train_loader), len(val_loader)

(360, 40)

In [13]:
test_dataset = SatelliteDataset(
feature_dir=feature_tiles_test,
label_dir=label_tiles_test,
weight_dir=None,
tiles=range(0, 100),
mu=None,
sigma=None,
sample=None
)
test_loader=DataLoader(test_dataset, batch_size, shuffle)


In [14]:
print(len(test_loader))

25


# Experiment

In [15]:
def save_mask(segmentation_mask, filename):
    segmentation_mask_np = segmentation_mask.detach().numpy()[0,:,:]
    segmentation_mask_np_uint8 = (segmentation_mask_np * 255).astype(np.uint8)
    segmentation_mask_pil = Image.fromarray(segmentation_mask_np_uint8)
    segmentation_mask_pil.save(filename)

import os
def mkpath(path: str) -> None:
    if not os.path.exists(path):
        os.makedirs(path)

In [16]:
class Experiment(pl.LightningModule):
    def __init__(self, arch="UNet", encoder_name="resnet34", encoder_weights="imagenet", in_channels=4, out_classes=1, experiment_name="Experiment1", loss="bce"):
        super().__init__()
        self.save_hyperparameters()
        self.experiment_name=experiment_name
        # Create Model
        self.model=smp.create_model(
            arch,
            encoder_name=encoder_name,
            encoder_weights=encoder_weights,
            in_channels=in_channels,
            classes=out_classes
        )

        self.loss=self._get_loss(loss)

        # Metrics
        self.val_miou = MIoU(2)
        self.val_acc = torchmetrics.Accuracy(task="binary")
        self.val_precision = torchmetrics.Precision(task="binary")
        self.val_recall = torchmetrics.Recall(task="binary")

        # test_Metrics
        self.test_miou=MIoU(2)
        self.test_acc=torchmetrics.Accuracy(task="binary")
        self.test_precision=torchmetrics.Precision(task="binary")
        self.test_recall=torchmetrics.Recall(task="binary")

    def configure_optimizers(self):
        optimizer=torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss(y_hat, y)
        self.log('train_loss', loss.detach(), prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        # Forward Pass
        x, y = batch
        y_hat_loss = self(x)
        y_hat = torch.sigmoid(y_hat_loss)
        loss = self.loss(y_hat_loss, y)

        # Log Loss and Accuracy
        self.val_acc(y_hat, y)
        self.val_miou(y_hat, y)
        self.val_precision(y_hat, y)
        self.val_recall(y_hat, y)
        self.log('val_loss', loss, prog_bar=True, logger=True)
        self.log('val_acc', self.val_acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_precision', self.val_precision, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_recall', self.val_recall, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_miou', self.val_miou, on_step=False, on_epoch=True, prog_bar=True, logger=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = torch.sigmoid(self(x))

        # Log Loss and Accuracy
        self.test_acc(y_hat, y)
        self.test_miou(y_hat, y)
        self.test_precision(y_hat, y)
        self.test_recall(y_hat, y)
        self.log('test_acc', self.test_acc, logger=True)
        self.log('test_precision', self.test_precision, logger=True)
        self.log('test_recall', self.test_recall, logger=True)
        self.log('test_miou', self.test_miou, logger=True)

        # Save Prediction and Label Masks
        y = y[0,:,:,:].cpu()
        out = y_hat[0,:,:,:].cpu()
        mkpath(f"predictions/{self.experiment_name}/masks")
        mkpath(f"predictions/{self.experiment_name}/preds")
        save_mask(y.cpu(), f"predictions/{self.experiment_name}/masks/mask_{batch_idx}.png")
        save_mask(out.round(), f"predictions/{self.experiment_name}/preds/pred_{batch_idx}.png")


    @staticmethod
    def _get_loss(loss):
        if loss == "dice":
            return dice_loss_with_logits
        elif loss == "dice_bce":
            return dice_bce_loss_with_logits
        return binary_cross_entropy_with_logits

# Training_Process

In [23]:
model=Experiment(experiment_name="Experiment1") #experiment0-epochs5

In [22]:
EPOCHS

50

In [24]:
trainer=pl.Trainer(max_epochs=EPOCHS, log_every_n_steps=1)
trainer.fit(
    model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
    )

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name           | Type            | Params | Mode 
-----------------------------------------------------------
0 | model          | Unet            | 24.4 M | train
1 | val_miou       | MIoU            | 0      | train
2 | val_acc        | BinaryAccuracy  | 0      | train
3 | val_precision  | BinaryPrecision | 0      | train
4 | val_recall     | BinaryRecall    | 0      | train
5 | test_miou      | MIoU            | 0      | train
6 | test_acc       | BinaryAccuracy  | 0      | train
7 | test_precision | BinaryPrecision | 0      | train
8 | test_recall    | BinaryRecall    | 0      | train
------------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:475: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


# Validation and test metrics

In [25]:
# run validation dataset
valid_metrics = trainer.validate(model, dataloaders=val_loader, verbose=False)
print(valid_metrics)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

[{'val_loss': 0.059950388967990875, 'val_acc': 0.985562801361084, 'val_precision': 0.8568124175071716, 'val_recall': 0.24338775873184204, 'val_miou': 0.6096869707107544}]


In [26]:
import numpy as np
from PIL import Image
# run test dataset
test_metrics = trainer.test(model, dataloaders=test_loader, verbose=False)
print(test_metrics)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:475: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

[{'test_acc': 0.9677057862281799, 'test_precision': 0.4401024281978607, 'test_recall': 0.6686097383499146, 'test_miou': 0.6641987562179565}]
